In [1]:
import os

import numpy as np
import gtsam # in conda env, $pip install gtsam 

from pytictoc import TicToc

In [2]:
"""
    helper functions 
"""

def vector6(x, y, z, a, b, c):
    """Create 6d double numpy array."""
    return np.array([x, y, z, a, b, c], dtype=float)

def info2mat(info):
    mat = np.zeros((6,6))
    ix = 0
    for i in range(mat.shape[0]):
        mat[i,i:] = info[ix:ix+(6-i)]
        mat[i:,i] = info[ix:ix+(6-i)]
        ix += (6-i)

    return mat

def read_g2o(fn):
    verticies, edges = [], []
    with open(fn) as f:
        for line in f:
            line = line.split()
            if line[0] == 'VERTEX_SE3:QUAT':
                v = int(line[1])
                pose = np.array(line[2:], dtype=np.float32)
                verticies.append([v, pose])

            elif line[0] == 'EDGE_SE3:QUAT':
                u = int(line[1])
                v = int(line[2])
                pose = np.array(line[3:10], dtype=np.float32)
                info = np.array(line[10:], dtype=np.float32)

                info = info2mat(info)
                edges.append([u, v, pose, info, line])

    return verticies, edges

def write_g2o(pose_graph, fn):
    import csv
    verticies, edges = pose_graph
    with open(fn, 'w') as f:
        writer = csv.writer(f, delimiter=' ')
        for (v, pose) in verticies:
            row = ['VERTEX_SE3:QUAT', v] + pose.tolist()
            writer.writerow(row)
        for edge in edges:
           writer.writerow(edge[-1])

def write_xyz(pose_graph, fn):
    import csv
    verticies, edges = pose_graph
    with open(fn, 'w') as f:
        writer = csv.writer(f, delimiter=' ')
        for (v, pose) in verticies:
            row = pose.tolist()
            writer.writerow(row)

In [3]:
"""
    noise models 
"""

# constants (user variables) 
priorModel = gtsam.noiseModel.Diagonal.Variances(vector6(1e-6, 1e-6, 1e-6, 1e-4, 1e-4, 1e-4))
odomModel = gtsam.noiseModel.Diagonal.Variances(vector6(1e-2, 1e-2, 1e-2, 1e-2, 1e-2, 1e-2))
loopModel = gtsam.noiseModel.Diagonal.Variances(vector6(0.5, 0.5, 0.5, 0.5, 0.5, 0.5))
robustLoopModel = gtsam.noiseModel.Robust.Create(gtsam.noiseModel.mEstimator.Cauchy.Create(1), loopModel)


# and helper functions 
def consecutive(node_idx_to, node_idx_from):
    return abs(node_idx_to - node_idx_from) == 1

def robustifyNoiseModel(noise_model, use_fixed=False):
    if use_fixed:
        return robustLoopModel
    else: 
        return gtsam.noiseModel.Robust.Create(gtsam.noiseModel.mEstimator.Cauchy.Create(1), noise_model)

In [4]:
def optimize_pgo(in_file, out_file, add_random_false_loops=[False, 0], use_robust_loops=True, use_fixed_noise=True):

    # load an initial graph and reconstruct it 
    is3D = True
    initial_graph, initial = gtsam.readG2o(in_file, is3D)

    graph = gtsam.NonlinearFactorGraph()
    for factor_idx in range(initial_graph.size()):
        factor = initial_graph.at(factor_idx)

        node_idx_from = factor.keys()[0]
        node_idx_to = factor.keys()[1]

        measurement = factor.measured()
    
        noise_model = factor.noiseModel()
        if use_fixed_noise:
            noise_model = odomModel            
 
        if use_robust_loops and not consecutive(node_idx_to, node_idx_from):
            noise_model = robustifyNoiseModel(noise_model, use_fixed_noise)

        graph.add( gtsam.BetweenFactorPose3(node_idx_from, node_idx_to, measurement, noise_model) )

    # add noise loops 
    if add_random_false_loops[0]:
        for ii in range(add_random_false_loops[1]):
            random_node_idx_from = np.random.randint(initial.size(), size=1)
            random_node_idx_to = np.random.randint(initial.size(), size=1)
            
            random_factor_idx = np.random.randint(initial_graph.size(), size=1)
            random_factor = graph.at(factor_idx)
            random_measurement = random_factor.measured()

            random_noise_model = random_factor.noiseModel()
            if use_fixed_noise:
                random_noise_model = odomModel            

            if use_robust_loops and not consecutive(node_idx_to, node_idx_from):
                random_noise_model = robustifyNoiseModel(random_noise_model, use_fixed_noise)

            graph.add( gtsam.BetweenFactorPose3(random_node_idx_from, random_node_idx_to, random_measurement, random_noise_model) )
     
    # add prior factor to avoid gauge problem 
    graph.add(gtsam.PriorFactorPose3(0, gtsam.Pose3(), priorModel))
      
    # optimizer 
    params = gtsam.LevenbergMarquardtParams()
    params.setVerbosity("Termination")  # this will show info about stopping conds
    optimizer = gtsam.LevenbergMarquardtOptimizer(graph, initial, params)

    # run opt 
    result = optimizer.optimize()
    print("Optimization complete")
    print("initial error = ", graph.error(initial))
    print("final error = ", graph.error(result))

    # save the optimized result 
    save_graph = gtsam.NonlinearFactorGraph()
    for factor_idx in range(graph.size()-1):
        factor = graph.at(factor_idx)

        node_idx_from = factor.keys()[0]
        node_idx_to = factor.keys()[1]

        measurement = factor.measured()
        noise_model = odomModel # because gtsam.writeG2o does not support to save robust factor, so this is a just, naive solution to save the node values.
 
        save_graph.add( gtsam.BetweenFactorPose3(node_idx_from, node_idx_to, measurement, noise_model) )
    
    print("Writing results to file: ", out_file)
    gtsam.writeG2o(save_graph, result, out_file)
    save_graph = read_g2o(out_file)
    write_xyz(save_graph, out_file + '.xyz')
    print ("Done!")



In [7]:
# tutorial datasets (see https://lucacarlone.mit.edu/datasets/)
data_dir = './data/'
seq_names = ['cubicle', 'grid3D', 'parking-garage', 'rim', 'sphere_bignoise_vertex3', 'torus3D']

# choose a sequence!
seq_name = seq_names[0] # change this 
in_file = data_dir + seq_name + '.g2o'

seq_save_dir = data_dir + seq_name
os.makedirs(seq_save_dir, exist_ok=True)

# save the initial one 
write_xyz(read_g2o(in_file), os.path.join(seq_save_dir, seq_name + '.xyz'))

"""
 main experiments 
"""
use_false_loops = True 
num_false_loops = 10

t = TicToc()

# Robust version
t.tic()
print('\nRobust version starts')
optimize_pgo(in_file=in_file,
             out_file=os.path.join(seq_save_dir, 'robust.g2o'), 
             add_random_false_loops=[use_false_loops, num_false_loops],
             use_robust_loops=True,
             use_fixed_noise=True)
t.toc()

# Non-robust version
t.tic()
print('\nNon-robust version starts')
optimize_pgo(in_file=in_file,
             out_file=os.path.join(seq_save_dir, 'nonrobust.g2o'), 
             add_random_false_loops=[use_false_loops, num_false_loops],
             use_robust_loops=False,
             use_fixed_noise=True)
t.toc()



Robust version starts
converged
errorThreshold: 68.7457271009 <? 0
absoluteDecrease: 0.000164157673424 <? 1e-05
relativeDecrease: 2.38789068582e-06 <? 1e-05
iterations: 10 >? 100
Optimization complete
initial error =  655.286328639572
final error =  68.74572710087696
Writing results to file:  ./data/cubicle/robust.g2o
Done!
Elapsed time is 2.737819 seconds.

Non-robust version starts
converged
errorThreshold: 13664.3463552 <? 0
absoluteDecrease: 0 <? 1e-05
relativeDecrease: 0 <? 1e-05
iterations: 78 >? 100
Optimization complete
initial error =  3306626.4190220158
final error =  13664.346355194586
Writing results to file:  ./data/cubicle/nonrobust.g2o
Done!
Elapsed time is 28.404976 seconds.
